<a href="https://colab.research.google.com/github/VioletRaven/COVID-19-Spark-Big-Data-Analysis/blob/main/QUERY_3_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

In [ ]:
#funziona ma non su colab
import locale
locale.setlocale(locale.LC_ALL, 'it_IT')

In [ ]:
import findspark

findspark.init()
findspark.find()

from pyspark import SparkContext
from datetime import datetime
from calendar import monthrange

In [ ]:
# Inizializzo lo Spark Context
sc = SparkContext("local[2]", "query_1.0")

Utilizzando somministrazioni-vaccini-summary-latest.csv, per il nord, centro e sud, determinare le tre aree (o regioni) che hanno registrato la percentuale piu` alta di popolazione vaccinata. Per il calcolo della statistica, considerare il totale del numero di prime dosi somministrate a partire dal 27 Dicembre 2020 e fino al 30 Giugno 2021. 

Inoltre, per calcolare la percentuale di popolazione vaccinata, considerare il file totale-popolazione.csv inviato via email, contenente per ogni regione il totale della popolazione stimata nel 2021.

Esempio di output:

Nord, Liguria, 54.19%
... altre 2 regioni del Nord... Centro, Umbria, 49.98%
... altre 2 regioni del Centro... Sud, Abruzzo, 50.71%
... altre 2 regioni del Sud...

Suggerimento: per determinare le regioni appartenenti al nord, centro e sud, e` possibile consultare liste definite staticamente.

In [ ]:
# secondo la suddivisione geografica fornita dall'ISTAT le 3 zone d'Italia (nord, centro, sud) sono così definite:

# NORD = Liguria, Lombardia, Piemonte, Valle d'Aosta, Emilia-Romagna, Friuli-Venezia Giulia, Trentino-Alto Adige, Veneto

# CENTRO = Lazio, Marche, Toscana, Umbria

# SUD = Abruzzo, Basilicata, Calabria, Campania, Molise, Puglia, Sardegna, Sicilia

In [ ]:
data = sc.textFile("/content/drive/MyDrive/Colab Notebooks/somministrazioni-vaccini-latest.csv")

#Identifico la prima riga che contiene le intestazioni e la elimino dall'RDD
header = data.first()
data = data.filter(lambda x: x != header).map(lambda line: line.split(','))

In [ ]:
data_2 = sc.textFile("/content/drive/MyDrive/Colab Notebooks/totale-popolazione.csv")

header_2 = data_2.first()
#remove empty lines
data_2 = data_2.filter(lambda x: x != header_2) \
         .filter(lambda x: x != '').map(lambda line : line.split(',')).map(lambda x: (x[0], int(x[1] )))  
        

In [ ]:
nord = ['Liguria', 'Lombardia', 'Piemonte', "Valle d'Aosta / Vallee d'Aoste", 'Emilia-Romagna', 'Friuli-Venezia Giulia', 'Provincia Autonoma Trento','Veneto', 'Provincia Autonoma Bolzano / Bozen']
centro = ['Lazio', 'Marche', 'Toscana', 'Umbria']
sud = ['Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Molise', 'Puglia', 'Sardegna', 'Sicilia']

regions = ['Liguria', 'Lombardia', 'Piemonte', "Valle d'Aosta / Vallee d'Aoste", 'Emilia-Romagna', 'Friuli-Venezia Giulia', 
           'Provincia Autonoma Trento','Veneto', 'Lazio', 'Marche', 'Toscana', 'Umbria', 'Provincia Autonoma Bolzano / Bozen',
           'Abruzzo', 'Basilicata', 'Calabria', 'Campania', 'Molise', 'Puglia', 'Sardegna', 'Sicilia']

def region_sorter(data):
  for i in regions:
    if i == data:
      if data in nord:
        return 'Nord'
      elif data in centro:
        return 'Centro'
      else:
        return 'Sud'

data_2 = data_2.map(lambda x: ( (x[0]), (region_sorter(x[0]), x[1])))


In [ ]:
# dal 27 Dicembre 2020 fino al 30 Giugno 2021.
data = data.map(lambda x: (  ( str( x[11] ), int( x[6] ) ) ) ).reduceByKey(lambda x, y: x + y)

In [ ]:
rdd = data_2.join(data)

In [ ]:
rdd_percentage = rdd.map(lambda x: (x[1][0][0], x[0], round((x[1][1] / x[1][0][1] ) * 100)))

[('Nord', 'Emilia-Romagna', 52),
 ('Nord', 'Liguria', 54),
 ('Nord', 'Provincia Autonoma Bolzano / Bozen', 48),
 ('Sud', 'Molise', 57),
 ('Nord', 'Provincia Autonoma Trento', 46)]

In [ ]:
sorted_rdd = rdd_percentage.map(lambda x:  (x[2],  ( x[0], x[1]  ) ) ).sortBy(lambda x: x[0], ascending = False)

In [ ]:
from heapq import nlargest

final_rdd = sorted_rdd.groupBy(lambda x: x[1][0]).flatMap(lambda k: nlargest(3, k[1], key=lambda x: x[0]))
final_rdd = final_rdd.map(lambda x: ( x[1][0], x[1][1], x[0]))

In [ ]:
final_rdd.coalesce(1).saveAsTextFile('/content/drive/MyDrive/Colab Notebooks/output_6')

In [ ]:
sc.stop()